In [1]:
import numpy as np
import dicom
from tkinter import *
from tkinter import filedialog
from tkinter import Tk

In [2]:
root = Tk()
root.withdraw()
file = filedialog.askopenfilename()
plan = dicom.read_file(file)

In [15]:
def GetGantry_and_MLC(plan):
    ##I'm going to start over down here, Mahmoud, and let's summarize what you've been doing so far
    #1. This code here is going to find the cardinal angles that need to be measured.
    #Cardinal angles
    # Gantry angle of start and end cps
    Start = plan.BeamSequence[0].ControlPointSequence[0].GantryAngle
    End = plan.BeamSequence[0].ControlPointSequence[-1].GantryAngle
    #convert to Varian Scale
    if(Start <=180):
        Start= 180 - Start
    else:
        Start = 540-Start
    if(End <=180):
        End = 180 - End
    else:
        End = 540-End
    #print(Start,End)
    possible_cardinals = np.array([Start,0,90,180,270,End])
    #cards= [x in np.arange(Start,End) for x in possible_cardinals]
    #this is only for CC, With CW the line will have reversed comparison operators.
    cards =  possible_cardinals[np.where(np.logical_and(possible_cardinals>=Start, possible_cardinals<=End))]
    print(cards)
    #2. Then find a list of control points that correspond to the surrounding control points at these angles.
    #this time I"ll do the icp calculation inside this loop.
    mlc_new_positions = {} #this is how to create a dictionary
    icp = []
    #looping over the control points to find the cardinal Angles "Varian Coordinates".
    for i in np.arange(0,len(plan.BeamSequence[0].ControlPointSequence)-1):
        current_gantry = plan.BeamSequence[0].ControlPointSequence[i].GantryAngle
        next_gantry = plan.BeamSequence[0].ControlPointSequence[i+1].GantryAngle
        #conversion to Varian Scale.
        current_gantry = 180-current_gantry if current_gantry <= 180 else 540-current_gantry
        next_gantry = 180 -next_gantry if next_gantry <=180 else 540-next_gantry
        for j in cards:
            #print(current_gantry, next_gantry, j)
            #print(np.linspace(current_gantry,next_gantry,np.absolute(current_gantry-next_gantry)+1))
            if j in np.arange(current_gantry,next_gantry+1):
                print ("The cardinal Beam angle in between: ",current_gantry, " and ", next_gantry, " degree 'Varian coordinate'  between control points: ", plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex, " and ", plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                print("Corresponds to ", str(j))
                #determine MLC positions of current and next control points.
                prev_mlc_pos= plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence[-1].LeafJawPositions
                next_mlc_pos = plan.BeamSequence[0].ControlPointSequence[i+1].BeamLimitingDevicePositionSequence[-1].LeafJawPositions
                #this array is just to calculate the interpolation. 
                temp_mlc = []
                #print(plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence)
                #interpolate to find the exact control point at the cardinal angle
                if j == current_gantry:
                    icp.append(float(plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex))
                    #print('Current Gantry!')
                    temp_mlc = [float(x) for x in plan.BeamSequence[0].ControlPointSequence[i].BeamLimitingDevicePositionSequence[-1].LeafJawPositions]
                    print('Writing temp MLC for Current Gantry.')
                    #write to the dictionary the temp_mlc with the gantry angle in Varian Standard as the key.
                    mlc_new_positions[j]=temp_mlc
                    #print (plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex)
                elif j == next_gantry:#ahhh you got me on this one was supposed to be an else if
                    icp.append(float(plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex))
                    #print (plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                    temp_mlc = [float(x) for x in plan.BeamSequence[0].ControlPointSequence[i+1].BeamLimitingDevicePositionSequence[-1].LeafJawPositions]
                    print('Next Gantry!')
                    mlc_new_positions[j] = temp_mlc
                else:
                    print('Interp gantry')
                    icp_temp = plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex*(1-((j-current_gantry)/(next_gantry-current_gantry)))+plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex*(1-((next_gantry-j)/(next_gantry-current_gantry)))
                    cp_index = float(plan.BeamSequence[0].ControlPointSequence[i].ControlPointIndex)
                    cp_index_next = float(plan.BeamSequence[0].ControlPointSequence[i+1].ControlPointIndex)
                    icp.append(icp_temp)
                    #loop through all MLC positions
                    print('checking mlc positions')
                    print('prev, next, interp')#uncomment the printed line below to actually get these values.
                    for x in np.arange(0,len(prev_mlc_pos)):
                        #y = y_0 + (x-x_0)*((y_1-y_0)/(x_1-x_0))
                        temp_interp = float(prev_mlc_pos[x])+(icp_temp-cp_index)*((float(next_mlc_pos[x])-float(prev_mlc_pos[x]))/(cp_index_next-cp_index))
                        temp_mlc.append(temp_interp)
                        #print(float(prev_mlc_pos[x]),next_mlc_pos[x], temp_interp)
                    mlc_new_positions[j] = temp_mlc
                print (mlc_new_positions[j])
                print('--------------------------')
    print('cardinal angles are: ', cards)
    print('control points are: ',icp)
    #print(mlc_new_positions)
    print ('Done!')
    return mlc_new_positions

In [19]:
MLC_Pos = GetGantry_and_MLC(plan)
print(type(MLC_Pos))

[   1.   90.  180.  270.  359.]
The cardinal Beam angle in between:  1.0  and  3.0  degree 'Varian coordinate'  between control points:  0  and  1
Corresponds to  1.0
Writing temp MLC for Current Gantry.
[-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 7.91, 5.35, 4.71, 3.97, 4.96, 5.65, 8.95, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 19.55, 22.25, 22.55, 22.85, 22.55, 21.95, 18.55, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.2

In [20]:
for gantry,mlc_pos in MLC_Pos.items():
    #this has your gantry angles and your control points.
    print(gantry,mlc_pos)

1.0 [-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 7.91, 5.35, 4.71, 3.97, 4.96, 5.65, 8.95, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, 19.55, 22.25, 22.55, 22.85, 22.55, 21.95, 18.55, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25]
90.0 [-0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -0.25, -5.125, -